In [1]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import numpy as np
import sys

sys.path.append(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\global-coastal-atlas\notebooks\01. utils')

from plot_funcs import future_shorelinepositions


In [2]:
# load in and keep only sandy transects
path_annual = r'P:\1000545-054-globalbeaches\03_Global_shorelines_update_2020\Transect_Mapping\Version_16102018\CSV\ShorelineMonitor_1984_2021_v1.5_set1.csv'
df_annual = pd.read_csv(path_annual)
#df_annual = df_annual[df_annual['flag_sandy'] == True]

df_annual = gpd.GeoDataFrame(df_annual, geometry= gpd.points_from_xy(df_annual.Intersect_lon, df_annual.Intersect_lat)).set_crs('epsg:3587')
df_annual = df_annual.to_crs('+proj=wintri')

df_annual.columns

Index(['transect_id', 'country_id', 'continent', 'country_name', 'changerate',
       'changerate_unc', 'flag_sandy', 'no_shorelines', 'dt', 'dist', 'RMSE',
       'outliers_1', 'outliers_2', 'Timespan', 'intercept', 'intercept_unc',
       'Start_lon', 'Start_lat', 'Intersect_lon', 'Intersect_lat', 'End_lon',
       'End_lat', 'coastline_idint', 'no_sedcomp', 'low_detect_shlines',
       'err_changerate', 'err_timespan', 'geometry'],
      dtype='object')

#### GHG emmisions (Vousdoukas, 2021)

In [3]:
basepath_vous = r'P:\1000545-054-globalbeaches\01_Global_Future_Shorelines\globalErosionProjectionsDataset_linear'

#RCP4.5 2050
df_vous_45_2050 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP45_2050.csv'), header= None)
df_vous_45_2050 = df_vous_45_2050.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_45_2050', 3: 'p5_45_2050', 
                                                   4: 'p17_45_2050', 5: 'p50_45_2050', 6: 'p83_45_2050', 7: 
                                                   'p95_45_2050', 8: 'p99_45_2050'})
df_vous_45_2050 = df_vous_45_2050.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')

#RCP4.5 2100
df_vous_45_2100 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP45_2100.csv'), header= None)
df_vous_45_2100 = df_vous_45_2100.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_45_2100', 3: 'p5_45_2100', 
                                                   4: 'p17_45_2100', 5: 'p50_45_2100', 6: 'p83_45_2100', 7: 
                                                   'p95_45_2100', 8: 'p99_45_2100'})
df_vous_45_2100 = df_vous_45_2100.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')

#RCP8.5 2050
df_vous_85_2050 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP85_2050.csv'), header= None)
df_vous_85_2050 = df_vous_85_2050.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_85_2050', 3: 'p5_85_2050', 
                                                   4: 'p17_85_2050', 5: 'p50_85_2050', 6: 'p83_85_2050', 7: 
                                                   'p95_85_2050', 8: 'p99_85_2050'})
df_vous_85_2050 = df_vous_85_2050.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')

#RCP8.5 2100
df_vous_85_2100 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP85_2100.csv'), header= None)
df_vous_85_2100 = df_vous_85_2100.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_85_2100', 3: 'p5_85_2100', 
                                                   4: 'p17_85_2100', 5: 'p50_85_2100', 6: 'p83_85_2100', 7: 
                                                   'p95_85_2100', 8: 'p99_85_2100'})
df_vous_85_2100 = df_vous_85_2100.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')

df_vous_45_2050.head(2)

,latitude,longitude,p1_45_2050,p5_45_2050,p17_45_2050,p50_45_2050,p83_45_2050,p95_45_2050,p99_45_2050
0,-50.378,-74.386,-2.0775,-1.6823,-1.2463,-0.744,-0.33039,-0.18465,-0.13225
1,-50.386,-74.303,-2.0773,-1.6899,-1.2751,-0.744,-0.34310,-0.18249,-0.12239


In [4]:
[set(df_vous_45_2050['latitude']).difference(x) for x in [set(df_vous_45_2100['latitude']), set(df_vous_85_2050['latitude']), set(df_vous_85_2100['latitude'])]]


[set(), set(), set()]

In [5]:
dfs_vous = [df_vous_45_2100, df_vous_85_2050, df_vous_85_2100]

df_vous = df_vous_45_2050.copy()
for dfv in dfs_vous:
    df_vous = df_vous.merge(dfv, on = ['latitude', 'longitude'], how = 'left')

assert len(df_vous_45_2050) == len(df_vous)

In [6]:
gdf_vous = gpd.GeoDataFrame(df_vous, geometry= gpd.points_from_xy(df_vous.longitude, df_vous_85_2100.latitude)).set_crs('epsg:3587')
gdf_vous = gdf_vous.to_crs('+proj=wintri')

##### Spatial Join on nearest transect

In [7]:
gdf_vous_join = gdf_vous.sjoin_nearest(df_annual, how = 'left', max_distance = 1000)
gdf_vous_join.head(2)


,latitude,longitude,p1_45_2050,p5_45_2050,p17_45_2050,p50_45_2050,p83_45_2050,p95_45_2050,p99_45_2050,p1_45_2100,...,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat,coastline_idint,no_sedcomp,low_detect_shlines,err_changerate,err_timespan
0,-50.378,-74.386,-2.0775,-1.6823,-1.2463,-0.744,-0.33039,-0.18465,-0.13225,-5.2368,...,-50.379144,-74.387125,-50.384042,-74.391782,-50.388940,5389.0,1.0,1.0,1.0,1.0
1,-50.386,-74.303,-2.0773,-1.6899,-1.2751,-0.744,-0.34310,-0.18249,-0.12239,-4.7864,...,-50.379169,-74.307625,-50.377870,-74.316375,-50.376571,5389.0,1.0,1.0,1.0,1.0


In [8]:
gdf_vous_join['p5_45_2050'].isnull().any()

False

In [9]:
gdf_vous_join['transect_id'].isnull().any()

False

In [10]:
def future_timeseries(a, b, cor_2050, cor_2100):
    a = 0
    #dt = np.arange(1984, 2101, 1)

    y2021 = a + (2021 - 1984) * b
    y2050 = a + (2050 - 1984) * b + cor_2050
    y2100 = a + (2100 - 1984) * b + cor_2100

    #s = pd.Series(index = dt, dtype='float64')
    # s.loc[1984] = a
    # s.loc[2021] = y2021
    # s.loc[2050] = y2050
    # s.loc[2100] = y2100

    y = [a, y2021, y2050, y2100]
    #s = s.interpolate(method = 'index').values

    return y

In [11]:
# AMBIENT
gdf_vous_join['Ambient'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, 0, 0), axis = 1)

# RCP4.5
gdf_vous_join['RCP4.5_p5'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p5_45_2050, row.p5_45_2100), axis = 1)
gdf_vous_join['RCP4.5_p50'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p50_45_2050, row.p50_45_2100), axis = 1)
gdf_vous_join['RCP4.5_p95'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p95_45_2050, row.p95_45_2100), axis = 1)

# RCP8.5
gdf_vous_join['RCP8.5_p5'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p5_85_2050, row.p5_85_2100), axis = 1)
gdf_vous_join['RCP8.5_p50'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p50_85_2050, row.p50_85_2100), axis = 1)
gdf_vous_join['RCP8.5_p95'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p95_85_2050, row.p95_85_2100), axis = 1)

In [12]:
gdf_vous_join.head(2)

,latitude,longitude,p1_45_2050,p5_45_2050,p17_45_2050,p50_45_2050,p83_45_2050,p95_45_2050,p99_45_2050,p1_45_2100,...,low_detect_shlines,err_changerate,err_timespan,Ambient,RCP4.5_p5,RCP4.5_p50,RCP4.5_p95,RCP8.5_p5,RCP8.5_p50,RCP8.5_p95
0,-50.378,-74.386,-2.0775,-1.6823,-1.2463,-0.744,-0.33039,-0.18465,-0.13225,-5.2368,...,1.0,1.0,1.0,"[0, 256.6827524645527, 457.8665314232562, 804....","[0, 256.6827524645527, 456.1842314232562, 800....","[0, 256.6827524645527, 457.1225314232562, 802....","[0, 256.6827524645527, 457.68188142325624, 804...","[0, 256.6827524645527, 455.8797314232562, 797....","[0, 256.6827524645527, 456.9316514232562, 801....","[0, 256.6827524645527, 457.6780214232562, 804...."
1,-50.386,-74.303,-2.0773,-1.6899,-1.2751,-0.744,-0.34310,-0.18249,-0.12239,-4.7864,...,1.0,1.0,1.0,"[0, 39.976307007810426, 71.30908817609428, 125...","[0, 39.976307007810426, 69.61918817609428, 121...","[0, 39.976307007810426, 70.56508817609428, 123...","[0, 39.976307007810426, 71.12659817609428, 124...","[0, 39.976307007810426, 69.36148817609428, 117...","[0, 39.976307007810426, 70.37420817609427, 122...","[0, 39.976307007810426, 71.11315817609427, 124..."


In [13]:
gdf_vous_join.columns

Index(['latitude', 'longitude', 'p1_45_2050', 'p5_45_2050', 'p17_45_2050',
       'p50_45_2050', 'p83_45_2050', 'p95_45_2050', 'p99_45_2050',
       'p1_45_2100', 'p5_45_2100', 'p17_45_2100', 'p50_45_2100', 'p83_45_2100',
       'p95_45_2100', 'p99_45_2100', 'p1_85_2050', 'p5_85_2050', 'p17_85_2050',
       'p50_85_2050', 'p83_85_2050', 'p95_85_2050', 'p99_85_2050',
       'p1_85_2100', 'p5_85_2100', 'p17_85_2100', 'p50_85_2100', 'p83_85_2100',
       'p95_85_2100', 'p99_85_2100', 'geometry', 'index_right', 'transect_id',
       'country_id', 'continent', 'country_name', 'changerate',
       'changerate_unc', 'flag_sandy', 'no_shorelines', 'dt', 'dist', 'RMSE',
       'outliers_1', 'outliers_2', 'Timespan', 'intercept', 'intercept_unc',
       'Start_lon', 'Start_lat', 'Intersect_lon', 'Intersect_lat', 'End_lon',
       'End_lat', 'coastline_idint', 'no_sedcomp', 'low_detect_shlines',
       'err_changerate', 'err_timespan', 'Ambient', 'RCP4.5_p5', 'RCP4.5_p50',
       'RCP4.5_p95', 'R

In [14]:
# gdf_vous_join = gdf_vous_join[['transect_id', 'country_id', 'continent', 'country_name',
#                                 'Ambient', 'RCP4.5_p5', 'RCP4.5_p50', 'RCP4.5_p95', 'RCP8.5_p5', 
#                                 'RCP8.5_p50', 'RCP8.5_p95']]

In [15]:
gdf_vous_join

,latitude,longitude,p1_45_2050,p5_45_2050,p17_45_2050,p50_45_2050,p83_45_2050,p95_45_2050,p99_45_2050,p1_45_2100,...,low_detect_shlines,err_changerate,err_timespan,Ambient,RCP4.5_p5,RCP4.5_p50,RCP4.5_p95,RCP8.5_p5,RCP8.5_p50,RCP8.5_p95
0,-50.378,-74.386,-2.0775,-1.6823,-1.2463,-0.744,-0.33039,-0.18465,-0.13225,-5.2368,...,1.0,1.0,1.0,"[0, 256.6827524645527, 457.8665314232562, 804....","[0, 256.6827524645527, 456.1842314232562, 800....","[0, 256.6827524645527, 457.1225314232562, 802....","[0, 256.6827524645527, 457.68188142325624, 804...","[0, 256.6827524645527, 455.8797314232562, 797....","[0, 256.6827524645527, 456.9316514232562, 801....","[0, 256.6827524645527, 457.6780214232562, 804...."
1,-50.386,-74.303,-2.0773,-1.6899,-1.2751,-0.744,-0.34310,-0.18249,-0.12239,-4.7864,...,1.0,1.0,1.0,"[0, 39.976307007810426, 71.30908817609428, 125...","[0, 39.976307007810426, 69.61918817609428, 121...","[0, 39.976307007810426, 70.56508817609428, 123...","[0, 39.976307007810426, 71.12659817609428, 124...","[0, 39.976307007810426, 69.36148817609428, 117...","[0, 39.976307007810426, 70.37420817609427, 122...","[0, 39.976307007810426, 71.11315817609427, 124..."
2,-50.485,-74.180,-110.9400,-92.7690,-71.3430,-44.640,-22.62700,-15.61000,-13.01300,-276.0600,...,1.0,1.0,1.0,"[0, 3.8445445286552458, 6.857836186249898, 12....","[0, 3.8445445286552458, -85.91116381375011, -2...","[0, 3.8445445286552458, -37.7821638137501, -99...","[0, 3.8445445286552458, -8.752163813750101, -2...","[0, 3.8445445286552458, -101.6121638137501, -4...","[0, 3.8445445286552458, -49.235163813750106, -...","[0, 3.8445445286552458, -12.370163813750104, -..."
3,-50.482,-74.181,-112.3300,-93.7550,-72.2410,-44.640,-23.58500,-15.64700,-13.58700,-270.0300,...,1.0,1.0,1.0,"[0, 5.889157039416696, 10.50498282706762, 18.4...","[0, 5.889157039416696, -83.25001717293237, -20...","[0, 5.889157039416696, -34.13501717293238, -93...","[0, 5.889157039416696, -5.1420171729323805, -1...","[0, 5.889157039416696, -106.97501717293238, -4...","[0, 5.889157039416696, -45.58801717293238, -16...","[0, 5.889157039416696, -7.813017172932382, -36..."
4,-50.480,-74.181,-113.3700,-93.3460,-72.3150,-44.640,-23.27700,-15.80400,-13.28900,-287.5700,...,1.0,1.0,1.0,"[0, 102.32099104799883, 182.518524572106, 320....","[0, 102.32099104799883, 89.172524572106, 88.42...","[0, 102.32099104799883, 137.87852457210602, 20...","[0, 102.32099104799883, 166.714524572106, 283....","[0, 102.32099104799883, 72.858524572106, -89.3...","[0, 102.32099104799883, 126.425524572106, 135....","[0, 102.32099104799883, 164.150524572106, 266...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504810,64.019,36.764,-81.6150,-71.2830,-54.4340,-35.010,-18.35600,-11.12800,-9.01280,-184.7900,...,1.0,1.0,1.0,"[0, -12.254560122596638, -21.859485624091302, ...","[0, -12.254560122596638, -93.14248562409131, -...","[0, -12.254560122596638, -56.8694856240913, -1...","[0, -12.254560122596638, -32.987485624091306, ...","[0, -12.254560122596638, -166.05948562409128, ...","[0, -12.254560122596638, -96.4474856240913, -2...","[0, -12.254560122596638, -43.997485624091304, ..."
504811,64.057,36.765,-81.1330,-66.9780,-53.2200,-34.112,-18.61600,-12.17500,-9.93060,-193.3000,...,1.0,1.0,1.0,"[0, -1.2985951551274446, -2.316412979416523, -...","[0, -1.2985951551274446, -69.29441297941652, -...","[0, -1.2985951551274446, -36.42841297941653, -...","[0, -1.2985951551274446, -14.491412979416523, ...","[0, -1.2985951551274446, -137.82641297941652, ...","[0, -1.2985951551274446, -74.99141297941652, -...","[0, -1.2985951551274446, -21.570412979416524, ..."
504812,64.700,39.880,-81.7060,-68.7700,-55.4620,-35.010,-18.89100,-12.07000,-9.96390,-187.6700,...,1.0,1.0,1.0,"[0, -1.9455689669374272, -3.4704743734559513, ...","[0, -1.9455689669374272, -72.24047437345595, -...","[0, -1.9455689669374272, -38.48047437345595, -...","[0, -1.9455689669374272, -15.540474373455952, ...","[0, -1.9455689669374272, -148.87047437345595, ...","[0, 

In [16]:
gdf_vous_filt = gdf_vous_join[['transect_id', 'country_id', 'continent', 'country_name', 'changerate', 'intercept',
       'Start_lon', 'Start_lat', 'Intersect_lon', 'Intersect_lat', 'End_lon',
       'End_lat',  'Ambient', 'RCP4.5_p5', 'RCP4.5_p50', 'RCP4.5_p95',
       'RCP8.5_p5', 'RCP8.5_p50', 'RCP8.5_p95']]

In [18]:
savefile = r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\03_Shorelinemonitor_future\future_shorelines.csv'

gdf_vous_filt.to_csv(savefile, index = False)

In [ ]:
import pandas as pd
df_fut = pd.read_csv(savefile)

: 

In [ ]:
box = 'BOX_187_084_11'

future_shorelinepositions(df_fut = df_fut, df_annual = df_annual, box = box)

: 

In [ ]:
box = 'BOX_187_084_11'

future_shorelinepositions(df_fut = df_fut, df_annual = df_annual, box = box)

: 

In [ ]:
box = 'BOX_187_090_49'

future_shorelinepositions(df_fut = df_fut, df_annual = df_annual, box = box)

: 

: 